# IBM Db2 Event Store - Machine Learning Modeling and Model Deployment 
IBM Db2 Event Store is a hybrid transactional/analytical processing (HTAP) system. This notebook illustrates the machine learning modeling and model deployment using IBM Db2 Event Store.

***Pre-Req: Event Store Data Analytics***

When finish this demo, you will learn:
- How to build a machine learning model
- How to save and deploy the model
- How to make realtime predictions with the deployed model

## Setting basic import clauses used by this notebook

In [1]:
from eventstore.oltp import EventContext
from eventstore.sql import EventSession
from pyspark.sql import SparkSession
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Setting the IP address to connect to your IBM Db2 Event Store cluster

For this, you will need to find out the connection string to your IBM Db2 Event Store cluster.

Perform the following steps:

- Replace the IP address in the below program code with the IP address of your local host
- Then execute the program cell below. It will connect to the IBM Db2 Event Store cluster in the provided connection string. 

In [2]:
from eventstore.common import ConfigurationReader

ip = "9.30.167.102"

endpoint = ip + ':1101'

print("Endpoint: "+ endpoint)

ConfigurationReader.setConnectionEndpoints(endpoint)

Endpoint: 9.30.167.102:1101


## Opening a database
The following code is used to open a database to be able to access its tables and data.

Run the command in the next program cell to define the database name. 

In [3]:
dbName = "TESTDB"

To run Spark SQL queries, you must set up a Db2 Event Store Spark session. The EventSession class extends the optimizer of the SparkSession class.

In [4]:
sparkSession = SparkSession.builder.appName("EventStore SQL in Python").getOrCreate()
eventSession = EventSession(sparkSession.sparkContext, dbName)

Now you can execute the command to open the database in the event session you created:

In [5]:
eventSession.open_database()

## Access an existing table in the database
The following code section retrieves the names of all tables that exist in the database.

In [6]:
with EventContext.get_event_context(dbName) as ctx:
   print("Event context successfully retrieved.")

table_names = ctx.get_names_of_tables()
for idx, name in enumerate(table_names):
   print(name)

Event context successfully retrieved.
IOT_TEMP


Now we have the name of the existing table. We then load the corresponding table and get the data frame references to access the table with query. The following code loads the tables and creates a temp view with the same name as the table.

In [7]:
tabName = "IOT_TEMP"

In [8]:
tab = eventSession.load_event_table(tabName)
tab.createOrReplaceTempView(tabName)
print("Table "+tabName+" successfully loaded and temp view created.")

Table IOT_TEMP successfully loaded and temp view created.


The next code retrieves the schema of the table we want to investigate:

In [9]:
try:
    resolved_table_schema = ctx.get_table(tabName)
    print(resolved_table_schema)
except Exception as err:
    print("Table not found")

ResolvedTableSchema(tableName=IOT_TEMP, schema=StructType(List(StructField(deviceID,IntegerType,false),StructField(sensorID,IntegerType,false),StructField(ts,LongType,false),StructField(ambient_temp,DoubleType,false),StructField(power,DoubleType,false),StructField(temperature,DoubleType,false))), sharding_columns=[u'deviceID', u'sensorID'], pk_columns=[u'deviceID', u'sensorID', u'ts'], partition_columns=None)


## Machine Learning Modeling
This section shows how to build a machine learning model with the data stored in the IBM Db2 Event Store database.

### Recall from the *Event_Store_Data_Analytics* notebook
- There are two input variables: ambient temperature and power consumption. The dependent variable is the sensor temperature reading.
- All features follow normal distribution.
- There is an obvious linear relationship between each independent variable and the dependent variable.

Now let's try generating a linear model to predict sensor temperature with power consumption and ambient temperature with the data stored in the IBM Db2 Event Store database table.

First import the relevant pyspark machine learning libraries:

In [10]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler 
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.evaluation import RegressionEvaluator

The following cell builds a new spark SQL dataframe from the `tab` dataframe, and prints out the `variable_df` dataframe schema.

In [11]:
variables = ["ambient_temp", "power"]
variable_df = tab.select(col("temperature").alias("label"), *variables)
variable_df.printSchema()

root
 |-- label: double (nullable = false)
 |-- ambient_temp: double (nullable = false)
 |-- power: double (nullable = false)



Now we split the dataframe into training set and test set at a percentage of 0.75 and 0.25.

We first build and train the model on the training set, then evaluate the model performance on the test set.

In [12]:
training, test = variable_df.randomSplit([0.75, 0.25], 42)

The model is built as a pipeline. There are three stages in the model pipeline, namely, *vector assembly*, *standarization*, and *model definition*. 

In the following cell we execute the three stages.

The training set is first assembled in to a dense dense vector. Then, the dense vector is standarized to a standard normal distribution. Finally the linear model is defined with regularization.

In [13]:
vectorAssembler = VectorAssembler(inputCols=variables, outputCol="unscaled_variables")
standardScaler = StandardScaler(inputCol="unscaled_variables", outputCol="features")
linear_model = LinearRegression(maxIter=10, regParam=.01)

stages = [vectorAssembler, standardScaler, linear_model]
pipeline = Pipeline(stages=stages)

The model is then trained on the training set. The trained model is used to make predictions on the test set.

In [14]:
model = pipeline.fit(training)
prediction = model.transform(test)

In the following cell we just show the first 10 rows out of the approximately 250 thousand in the prediction:

In [15]:
prediction.show(10)

+------------------+------------------+------------------+--------------------+--------------------+------------------+
|             label|      ambient_temp|             power|  unscaled_variables|            features|        prediction|
+------------------+------------------+------------------+--------------------+--------------------+------------------+
| 26.86771831781082|19.000636049568666|  4.69462969258635|[19.0006360495686...|[9.49493491610541...| 32.07430501284979|
|26.890434961120302|18.087410479036652|0.5006280784763337|[18.0874104790366...|[9.03858085862524...|28.799798039718354|
|27.241242822746088| 15.64429094179144| 8.926407054969541|[15.6442909417914...|[7.81771325514649...| 29.83166280881514|
| 28.29965792479851|16.952745077149896| 8.257884674094836|[16.9527450771498...|[8.47156962203477...|31.195010255051116|
|28.386549343567314|18.789370305272453| 3.920690375410091|[18.7893703052724...|[9.38936189808317...|  31.4146222945987|
| 28.55349560150172|17.485530835101855| 

### Model Evaluation
The performance of the linear model we just built can be evaluated using multiple error metrics.

We first load and define a regression evaluator using pyspark library.

In [16]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

We then evaluate the model performance with multiple error metrics.

In [17]:
rmse = evaluator.evaluate(prediction)

mae = evaluator.evaluate(prediction, {evaluator.metricName: "mae"})

r2 = evaluator.evaluate(prediction, {evaluator.metricName: "r2"})

Finally we put the error metrics into a dataframe to help visualization

In [18]:
error_df = {"r2":r2, "mae":mae, "rmse":rmse}
error_df = pd.DataFrame.from_dict(error_df, orient="index")
error_df.columns = ["error metrics"]

In [19]:
# Show error metrics
error_df

,error metrics
mae,1.195539
r2,0.800110
rmse,1.498069


**Model Summarization**  
The r2 metrics shows the percentage of the variance in the data that is explained by the model. Our model has a high r2 value that is very close to 1, meaning most of the variance in the test data can be explained with our model.

## Model Deployment
After the model is trained in the notebook, a user can save the model into Watson Studio Local by using the save function in the `dsx_ml` library. Then the model can be used to generate real time online scoring on the data streamed into IBM Db2 Event Store.

Users of IBM Watson Studio are allowed to save the spark, keras and scikit-learn models trained on the data stored in the IBM Db2 Event Store.

For more information, please visit [IBM Watson Studio Locao - Save Model](https://content-dsxlocal.mybluemix.net/docs/content/SSAS34_current/local-dev/ml-notebook-doc.htm)

The first step is to save the model with metadata

In [20]:
from dsx_ml.ml import save
model_name = "Event_Store_IOT_Sensor_Temperature_Prediction_Model"
saved_model = save(name=model_name, 
                   model=model,
                   test_data=test,
                   algorithm_type="Regression",
                   source='Event_Store_Modeling.ipynb',
                   description="Linear regression model to predict IOT sensor temperature"
                  )

Using TensorFlow backend.


With the saved model we then define a header that contains authorization, which will be sent to the endpoint, and then retrieve the endpoint to the saved model to allow us to externally access it. Note that the host name `dsxl-api` needs to be replaced with the corresponding external IP address of your IBM Watson Studio cluster.

In [21]:
import os
import requests

header_online = {'Content-Type': 'application/json', 'Authorization': os.environ['DSX_TOKEN']}
# Retrive the endpoint to the saved model
print(saved_model["scoring_endpoint"])

https://dsxl-api/v3/project/score/Python27/spark-2.0/db2%20event%20store/Event_Store_IOT_Sensor_Temperature_Prediction_Model/1


### Make a Prediction with the Deployed Model
Now the model has been saved and deployed. After deployment, the endpoint of model can be used to make a prediction for new data using the online scoring service.  

The following sample code snippet calls the scoring endpoint to make predictions on the new data. The prediction can be made on single datum, or on batch data.

First create a sample datum to be predicted by the model.

In [22]:
new_data = {"deviceID" : 2, "sensorID": 24, "ts": 1541430459386, "ambient_temp": 30, "power": 10}
new_data2 = {"deviceID" : 1, "sensorID": 12, "ts": 1541230400000, "ambient_temp": 16, "power": 50}

- Single datum prediction

In [23]:
payload_scoring = [new_data]
scoring_response = requests.post(saved_model["scoring_endpoint"], json=payload_scoring, headers=header_online, verify=False)

print(scoring_response.text)

{"success":true,"description":"Success","object":{"error":"","output":{"classes":[],"predictions":[48],"probabilities":[]},"returnCode":"0"}}


Because this is a regression model, we can retrieve the prediction

In [24]:
scoring_response.json()["object"]["output"]["predictions"]

[48]

- Batch prediction

In [25]:
payload_scoring = [new_data, new_data2]
scoring_response = requests.post(saved_model["scoring_endpoint"], json=payload_scoring, headers=header_online, verify=False)

print(scoring_response.text)

{"success":true,"description":"Success","object":{"error":"","output":{"classes":[],"predictions":[48,50],"probabilities":[]},"returnCode":"0"}}


In [26]:
scoring_response.json()["object"]["output"]["predictions"]

[48, 50]

### Online Scoring with Curl

Once the machine learning model is deployed. Online scoreing can also be performed in commandline with curl.  

```bash
To use curl for online scoring, user has to first obtain a authentication token from the cluster by running following command from the command line.

A json file will be returned with an "accessToken" that can be used
Example command is:

    curl -k -X GET "https://<HostIP>/v1/preauth/validateAuth" -u admin:password

With the authenticati on token, user can run the following commands from the command line for single or batch scoring:

#single scoring
    echo "Single Scoring" 
    curl -k -X POST "https://$CLUSTER_IP/v3/project/score/Python27/spark-2.0/IOT_demo/Event_Store_IOT_Sensor_Temperature_Prediction_Model/5"  -H 'Content-Type: application/json' -H "Authorization: Bearer $accessToken" -d ' { "deviceID" : 2, "sensorID": 24, "ts": 1541430459386, "ambient_temp": 30, "power": 10 }'

#batch scoring
    echo "Batch Scoring" 
    curl -k -X POST "https://$CLUSTER_IP/v3/project/score/Python27/spark-2.0/IOT_demo/Event_Store_IOT_Sensor_Temperature_Prediction_Model/5"  -H 'Content-Type: application/json' -H "Authorization: Bearer $accessToken" -d '[ { "deviceID" : 2, "sensorID": 24, "ts": 1541430459386, "ambient_temp": 30, "power": 10 }, {"deviceID" : 1, "sensorID": 12, "ts": 1541230400000, "ambient_temp": 16, "power": 50}]'

```
Please find the bash scripts demonstrating online scoring with curl in the `/curl` directory of this git repo. 

## Summary
This demo introduced you to the IBM Db2 Event Store API for Machine Learning and Model Deployment.